For each cell (or node),

 A* computes:


f(n)=g(n)+h(n)

Where:

g(n) → Cost from the start node to the current node

h(n) → Heuristic estimate of the cost from the current node to the goal (e.g., Euclidean distance)

f(n) → Total estimated cost of the path through node n

A* always expands the node with the lowest f-value first.

In [13]:
import math
import heapq






In [14]:
# Class to store details about each cell in the grid
class Node:
    def __init__(self):
        self.parent_row = 0   # Row index of the parent node
        self.parent_col = 0   # Column index of the parent node
        self.total_cost = float('inf')  # f = g + h
        self.cost_from_start = float('inf')  # g
        self.heuristic_cost = 0  # h

## Functions


In [15]:

# Check if a cell position is inside the grid
def is_inside_grid(row, col, ROWS, COLS):
    return (0 <= row < ROWS) and (0 <= col < COLS)





In [16]:
# Check if a cell is not blocked
def is_walkable(grid, row, col):
    return grid[row][col] == 0


In [17]:
# Check if this cell is the destination
def is_goal(row, col, goal):
    return row == goal[0] and col == goal[1]



In [18]:
# Heuristic function (Euclidean distance)
def calculate_heuristic(row, col, goal):
    return math.sqrt((row - goal[0])**2 + (col - goal[1])**2)



In [19]:
# Print the final path
def show_path(node_info, goal):
    print("\nPath found:")
    path = []
    row, col = goal

    # Move backwards from goal to start using parent links
    while not (node_info[row][col].parent_row == row and node_info[row][col].parent_col == col):
        path.append((row, col))
        temp_row = node_info[row][col].parent_row
        temp_col = node_info[row][col].parent_col
        row, col = temp_row, temp_col

    # Add the start node and reverse to get correct order
    path.append((row, col))
    path.reverse()

    for step in path:
        print("->", step, end=" ")
    print("\n")

###  A* Algorithm Steps

**1. Initialize**
- Put the start node in an open list (priority queue sorted by f).
- Mark all other nodes as unvisited.



**2. Loop until the goal is found**
1. Select the node with the smallest f(n) from the open list.  
2. Move it to the closed list (visited).  
3. For each of its neighbors:
   - If it’s blocked or already visited → skip.  
   - Compute new costs:  
     ```
     g_new = g_current + distance(current, neighbor)
     h_new = Euclidean_distance(neighbor, goal)
     f_new = g_new + h_new
     ```
   - If this path is better, update the neighbor’s info and push it into the open list.



**3. Goal Found**
- Stop when the goal node is reached.  
- Reconstruct the path by tracing back the parent nodes.


**4. No Path Found**
- If the open list is empty and the goal is not found → no valid path exists.


In [20]:
# ---------------------- A* Algorithm ----------------------

def a_star(grid, start, goal):
    ROWS = len(grid)
    COLS = len(grid[0])

    # Check start/goal validity
    if not is_inside_grid(start[0], start[1], ROWS, COLS) or not is_inside_grid(goal[0], goal[1], ROWS, COLS):
        print("Start or goal is outside the grid.")
        return

    if not is_walkable(grid, start[0], start[1]) or not is_walkable(grid, goal[0], goal[1]):
        print("Start or goal is blocked.")
        return

    if is_goal(start[0], start[1], goal):
        print("Already at the goal!")
        return

    # Initialize visited matrix
    visited = [[False for _ in range(COLS)] for _ in range(ROWS)]
    # Initialize node details
    node_info = [[Node() for _ in range(COLS)] for _ in range(ROWS)]

    # Set up start node
    start_row, start_col = start
    node_info[start_row][start_col].total_cost = 0.0
    node_info[start_row][start_col].cost_from_start = 0.0
    node_info[start_row][start_col].heuristic_cost = 0.0
    node_info[start_row][start_col].parent_row = start_row
    node_info[start_row][start_col].parent_col = start_col

    # Open list (priority queue based on total cost f)
    open_list = []
    heapq.heappush(open_list, (0.0, start_row, start_col))

    goal_found = False

    # 8 possible directions
    moves = [
        (0, 1), (0, -1), (1, 0), (-1, 0),  # 4 straight directions
        (1, 1), (1, -1), (-1, 1), (-1, -1)  # 4 diagonals
    ]

    while open_list:
        # Get node with smallest f value
        _, current_row, current_col = heapq.heappop(open_list)
        visited[current_row][current_col] = True

        # Explore neighbors
        for move in moves:
            neighbor_row = current_row + move[0]
            neighbor_col = current_col + move[1]

            # Skip invalid, blocked, or visited cells
            if not is_inside_grid(neighbor_row, neighbor_col, ROWS, COLS):
                continue
            if not is_walkable(grid, neighbor_row, neighbor_col):
                continue
            if visited[neighbor_row][neighbor_col]:
                continue

            # If goal found
            if is_goal(neighbor_row, neighbor_col, goal):
                node_info[neighbor_row][neighbor_col].parent_row = current_row
                node_info[neighbor_row][neighbor_col].parent_col = current_col
                print("Goal found ")
                show_path(node_info, goal)
                goal_found = True
                return

            # Compute costs
            new_g = node_info[current_row][current_col].cost_from_start + 1.0
            new_h = calculate_heuristic(neighbor_row, neighbor_col, goal)
            new_f = new_g + new_h

            # If this path is better, update the cell
            if node_info[neighbor_row][neighbor_col].total_cost == float('inf') or node_info[neighbor_row][neighbor_col].total_cost > new_f:
                heapq.heappush(open_list, (new_f, neighbor_row, neighbor_col))
                node_info[neighbor_row][neighbor_col].total_cost = new_f
                node_info[neighbor_row][neighbor_col].cost_from_start = new_g
                node_info[neighbor_row][neighbor_col].heuristic_cost = new_h
                node_info[neighbor_row][neighbor_col].parent_row = current_row
                node_info[neighbor_row][neighbor_col].parent_col = current_col

    if not goal_found:
        print("Failed to find a path ")



In [21]:
# ---------------------- Main ----------------------

def main():
    grid = [
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 1, 1, 0, 1, 0, 1, 0, 1, 0],
        [0, 1, 0, 0, 0, 1, 0, 1, 0, 1],
        [0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 1, 0, 1],
        [0, 1, 0, 1, 1, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    ]

    start = [0, 0]
    goal = [6, 8]

    a_star(grid, start, goal)


if __name__ == "__main__":
    main()


Goal found 

Path found:
-> (0, 0) -> (0, 1) -> (0, 2) -> (1, 3) -> (2, 4) -> (1, 5) -> (2, 6) -> (3, 7) -> (4, 8) -> (5, 7) -> (6, 8) 

